<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/testes_acesso_jira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependências



In [ ]:
!pip install jira > install_jira.log
!pip install sqlite3 > install_sqlite.log
!sudo apt install sqlite3 > install_sqlite3.log

# Apoio (Classes)

In [ ]:
from jira import JIRA
import os
from datetime import datetime, timedelta
import tqdm
import sqlite3
import subprocess
import sqlite3

JIRA_SERVER = 'https://issues.apache.org/jira'
DATABASE_NAME = "issues_db.db"

os.environ['DATABASE_NAME'] = DATABASE_NAME
# Credentials
os.environ['USERNAME'] = 'armandossrecife'
os.environ['PASSWORD'] = 'sky1979#ce'
username = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

class JiraIssue:
  def __init__(self, key, summary, issue_type, status, priority, description, comments):
    self.key = key
    self.summary = summary
    self.issue_type = issue_type
    self.status = status
    self.priority = priority
    self.description = description
    self.comments = comments

  def get_comments(self) -> dict:
    return self.comments

  def __str__(self):
    return (f'Key: {self.key}, Summary: {self.summary}, Type: {self.issue_type}, Status: {self.status}')

class JiraIssues:
  def __init__(self,project, issues):
    self.project = project
    self.issues = issues

  def add_issue(self, issue):
    self.issues.append(issue)

  def get_issues(self) -> list:
    return self.issues

  def update_issues(self, issues):
    self.issues = issues

  def __str__(self):
    str_issues = ""
    for issue in self.get_issues():
      str_issues = str_issues + str(issue)
      str_issues = str_issues + ', '
    str_issues = '[' + str_issues + ']'
    return (f'Project: {self.project}, Qdt of issues: {len(self.issues)}, Issues: {str_issues}')

# Classe de utilidades para manipular o servidor Jira
class JiraUtils:
  def __init__(self, project, jira_instance):
    self.project = project
    self.jira_jira_instance = jira_instance

  def generate_intervals_between_dates(self, date1: tuple, date2: tuple, distance=120) -> list:
    start_date = datetime(date1[0], date1[1], date1[2])
    end_date = datetime(date2[0], date2[1], date2[2])
    interval_days = distance
    # Initialize a list to store the intervals
    intervals = []
    # Initialize the current date as the start date
    current_date = start_date
    # Loop to generate intervals until the current date is less than or equal to the end date
    while current_date < end_date:
        interval = (current_date, current_date + timedelta(days=interval_days - 1))
        intervals.append(interval)
        current_date += timedelta(days=interval_days)
    return intervals

  def convert_interval_dates(self, dates: list) -> list:
    list_interval_dates = []
    for each in dates:
      date1 = each[0]
      # Convert the date to a string in the format "YYYY/MM/DD".
      str_date1 = date1.strftime("%Y/%m/%d")
      date2 = each[1]
      str_date2 = date2.strftime("%Y/%m/%d")
      elemento = str_date1, str_date2
      list_interval_dates.append(elemento)
    return list_interval_dates

  def generate_list_of_sentences(self, dates: list) -> list:
    lista_sentencas = []
    for each in dates:
      str_date1 = each[0].strftime("%Y/%m/%d")
      str_date2 = each[1].strftime("%Y/%m/%d")
      sentenca = f'project={self.project.upper()} and created>="{str_date1}" and created<="{str_date2}"'
      lista_sentencas.append(sentenca)
    return lista_sentencas

  def get_list_of_block_issues_by_dates(self,date1, date2, distance=120) -> list:
    print('Aguarde...')
    t1 = datetime.now()
    list_of_dates = self.generate_intervals_between_dates(date1,date2,distance)
    lista_sentencas = self.generate_list_of_sentences(list_of_dates)
    lista_bloco_issues_by_date = []
    total_items = len(lista_sentencas)
    i = 0
    iterable_lista_sentencas = tqdm.tqdm(lista_sentencas, total=total_items)
    for each in iterable_lista_sentencas:
      issues_by_date_temp = self.jira_jira_instance.search_issues(each,maxResults=1000)
      print(f'Range: {each}, qtd issues: {len(issues_by_date_temp)}')
      lista_bloco_issues_by_date.append(issues_by_date_temp)
      percentage = (i + 1) / total_items * 100
      iterable_lista_sentencas.set_description(f"Progress Message Analysis")
    i += 1
    t2 = datetime.now()
    print(t2)
    print(f'Tempo da consulta: {t2-t1}')
    return lista_bloco_issues_by_date

  def concatenate_block_of_issues(self,block_of_issues):
    concatenated_list = [item for sublist in block_of_issues for item in sublist]
    print(f'Total de issues recuperados: {len(concatenated_list)}')
    return concatenated_list

class IssuesDatabase:
    def __init__(self, database_name):
        self.database_name = database_name
        self.create_tables()

    def create_tables(self):
        self.conn = sqlite3.connect(self.database_name)
        self.cursor = self.conn.cursor()

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Issues (
                id INTEGER PRIMARY KEY,
                project TEXT,
                key TEXT,
                summary TEXT,
                issue_type TEXT,
                status TEXT,
                priority TEXT,
                description TEXT
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Comments (
                id INTEGER PRIMARY KEY,
                key TEXT,
                comment TEXT
            )
        ''')

        self.conn.commit()

    def insert_in_table_issues(self, project, key, summary, issue_type, status, priority, description):
        values = (None, project, key, summary, issue_type, status, priority, description)
        self.cursor.execute('''
            INSERT INTO Issues
            (id, project, key, summary, issue_type, status, priority, description)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', values)

        self.conn.commit()

    def insert_in_table_comments(self, key, comment):
        values = (None, key, comment)
        self.cursor.execute('''
            INSERT INTO Comments
            (id, key, comment)
            VALUES (?, ?, ?)
        ''', values)

        self.conn.commit()

    def show_content(self, table):
        query = f"SELECT * FROM {table}"
        self.cursor.execute(query)

        rows = self.cursor.fetchall()
        for row in rows:
            print(row)

    def show_n_lines(self, table, n):
        query = f"SELECT * FROM {table}"
        self.cursor.execute(query)

        rows = self.cursor.fetchall()

        for i, row in enumerate(rows):
            print(row)
            if i == n:
              break

    def close_connection(self):
        self.conn.close()

# Cria uma instância para acessar o servidor Jira

In [ ]:
# Create a JIRA client instance
jira_instance = JIRA(server=JIRA_SERVER, basic_auth=(username, password))

jira_utilidades = JiraUtils(project='CASSANDRA', jira_instance=jira_instance)

# Coleta os issues do projeto Cassandra

In [ ]:
cassandra_block_issues_by_date_2009_2023 = jira_utilidades.get_list_of_block_issues_by_dates(date1=(2009,3,2), date2=(2023,9,12), distance=120)

Aguarde...


Progress Message Analysis:   2%|▏         | 1/45 [00:02<02:05,  2.85s/it]

Range: project=CASSANDRA and created>="2009/03/02" and created<="2009/06/29", qtd issues: 263


Progress Message Analysis:   4%|▍         | 2/45 [00:06<02:15,  3.15s/it]

Range: project=CASSANDRA and created>="2009/06/30" and created<="2009/10/27", qtd issues: 251


...

Range: project=CASSANDRA and created>="2022/12/19" and created<="2023/04/17", qtd issues: 328


Progress Message Analysis:  98%|█████████▊| 44/45 [03:46<00:03,  3.52s/it]

Range: project=CASSANDRA and created>="2023/04/18" and created<="2023/08/15", qtd issues: 298


Progress Message Analysis: 100%|██████████| 45/45 [03:47<00:00,  5.06s/it]

Range: project=CASSANDRA and created>="2023/08/16" and created<="2023/12/13", qtd issues: 95
2023-09-16 17:13:42.408057
Tempo da consulta: 0:03:47.784661


In [ ]:
cassandra_issues_from_2009_to_2023 = jira_utilidades.concatenate_block_of_issues(block_of_issues=cassandra_block_issues_by_date_2009_2023)

Total de issues recuperados: 18579


## Popula um manipulador de Issues do Jira

In [ ]:
cassandra_issues = JiraIssues(project='Cassandra', issues=[])

print('Popula issues do Cassandra')
for i,issue in enumerate(cassandra_issues_from_2009_to_2023):
  dict_comments_issue = {}
  list_comments = []
  for comment in issue.fields.comment.comments:
    elemento = (comment.id, comment.body)
    list_comments.append(elemento)
  dict_comments_issue[issue.key] = list_comments
  # Cria um JiraIssue
  jira_issue = JiraIssue(key=issue.key, summary=issue.fields.summary, status=issue.fields.status, issue_type=issue.fields.issuetype,priority=issue.fields.priority,description=issue.fields.description, comments=dict_comments_issue)
  cassandra_issues.add_issue(jira_issue)
print(f'{len(cassandra_issues.get_issues())} issues populados com sucesso!')

Popula issues do Cassandra
18579 issues populados com sucesso!


In [ ]:
print('Mostra o conteúdo dos 10 primeiros issues...')
for i,each in enumerate(cassandra_issues.get_issues()):
  print(i+1, each)
  print(f'Description: {each.description}')
  for k,v in each.get_comments().items():
    print(k)
    print(f'Comments: {len(v)}')
    for j in v:
      print(j)
  if i==10:
    break

Mostra o conteúdo dos 10 primeiros issues...
1 Key: CASSANDRA-263, Summary: get_slice needs to support desc from last column, Type: Bug, Status: Resolved
Bug Resolved Normal
Description: At the moment there's no way to ask for a slice starting with the last column and going desc
CASSANDRA-263
Comments: 7
('12724762', "Is the problem that you don't know what the last column is? Normally, an application should be able to insert a fake column that's always larger than any real columns. Do you think that's good enough?")
('12724766', 'Yes.\n\nI was thinking that we could have the "CF" format (w/ no ":") stand for "start w/ first if asc or last if desc."  Reading the last from the column index should be reasonably efficient, right?')
('12725251', 'Using the column index to start with the last column group is possible and is efficient. I am just wondering if it is a good idea to give empty starting column different meanings depending on the ordering.')
('12725254', "I'm open to alternative A

# Carrega os issues coletados em um banco (Sqlite)

In [ ]:
db = IssuesDatabase("issues_db.db")
print('Popula o banco de dados issues_db...')
total_items = len(cassandra_issues.get_issues())
i = 0
iterable_cassandra_issues = tqdm.tqdm(cassandra_issues.get_issues(), total=total_items)
for each_jira_issue in iterable_cassandra_issues:
  db.insert_in_table_issues("CASSANDRA", each_jira_issue.key, each_jira_issue.summary, str(each_jira_issue.issue_type), str(each_jira_issue.status), str(each_jira_issue.priority), each_jira_issue.description)
  if len(each_jira_issue.get_comments().keys()) > 0:
    for issue_id,comments in each_jira_issue.get_comments().items():
      if len(comments) > 0:
        for comment in comments:
          if len(comment) > 0:
            elemento = comment[0] + ': ' + comment[1]
            db.insert_in_table_comments(issue_id, elemento)
  percentage = (i + 1) / total_items * 100
  iterable_cassandra_issues.set_description(f"Progress inserting data")
  i += 1
db.close_connection()
print('Banco issues_db carregado com sucesso!')

Progress inserting data:   0%|          | 2/18579 [00:00<30:22, 10.19it/s]

Popula o banco de dados issues_db...


Progress inserting data: 100%|██████████| 18579/18579 [28:24<00:00, 10.90it/s]

Banco issues_db carregado com sucesso!


In [ ]:
!ls -liath

total 101M
2752518 drwxr-xr-x 1 root root 4.0K Sep 16 19:52 .
3145952 -rw-r--r-- 1 root root 101M Sep 16 19:52 issues_db.db
3145950 drwxr-xr-x 2 root root 4.0K Sep 16 18:51 .ipynb_checkpoints
3145786 -rw-r--r-- 1 root root 1.8K Sep 16 17:08 install_jira.log
3145739 drwxr-xr-x 1 root root 4.0K Sep 16 17:07 ..
2752519 drwxr-xr-x 1 root root 4.0K Sep 14 13:23 sample_data
1703943 drwxr-xr-x 4 root root 4.0K Sep 14 13:22 .config


## Mostra um fragmento das tabelas Issues e Comments

In [ ]:
db = IssuesDatabase("issues_db.db")
db.show_n_lines(table='Issues', n=10)

(1, 'CASSANDRA', 'CASSANDRA-263', 'get_slice needs to support desc from last column', 'Bug', 'Resolved', 'Normal', "At the moment there's no way to ask for a slice starting with the last column and going desc")
(2, 'CASSANDRA', 'CASSANDRA-262', 'get_slice needs to allow returning all columns', 'Bug', 'Resolved', 'Normal', "Right now get_slice requires you to enter a 'large' value, -1 used to indicate 'all columns'. We should allow this.")
(3, 'CASSANDRA', 'CASSANDRA-261', 'get_slice needs offset + limit', 'Bug', 'Resolved', 'Normal', 'Right now get_slice does not allow you to provide an offset.  This would help for pagination.')
(4, 'CASSANDRA', 'CASSANDRA-260', 'Unable to read all columns in a column family from the CLI', 'Bug', 'Resolved', 'Normal', 'Since r788516 the typical \'get all columns for a row/cf\' example no longer works.\n\ncassandra> get Table1.Standard1[\'jsmith\']\nis the example from the wiki.\n\nThis is due to Table.getSliceFrom now requiring the cfName be split by "

In [ ]:
db.show_n_lines(table='Comments', n=10)

(1, 'CASSANDRA-263', "12724762: Is the problem that you don't know what the last column is? Normally, an application should be able to insert a fake column that's always larger than any real columns. Do you think that's good enough?")
(2, 'CASSANDRA-263', '12724766: Yes.\n\nI was thinking that we could have the "CF" format (w/ no ":") stand for "start w/ first if asc or last if desc."  Reading the last from the column index should be reasonably efficient, right?')
(3, 'CASSANDRA-263', '12725251: Using the column index to start with the last column group is possible and is efficient. I am just wondering if it is a good idea to give empty starting column different meanings depending on the ordering.')
(4, 'CASSANDRA-263', "12725254: I'm open to alternative APIs, but that one makes sense to me and doesn't require adding extra parameters :)")
(5, 'CASSANDRA-263', "12737354: Attach a fix. If startColumn is empty and isAscending is false, assume that we scan from the largest column in descen